In [1]:
import sys
import os
import pandas as pd
import spacy
import ast

%load_ext autoreload
%autoreload 2
PATH = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(PATH)

In [185]:
data = pd.read_csv(PATH + '/skills_matcher/data/data_final_raw.csv')
data.head(2)

,Unnamed: 0,ISCO,major_job,job,position,location,description,entities_auto_label,entities_manual_label
0,0,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': physics, 'label': 'KNOWLEDGE'}, {'..."
1,1,21,SCIENCE AND ENGINEERING PROFESSIONALS,physicist,Accelerator Physicist id54315,"Villigen PSI, Aargau",[' You have an academic degree in physics or e...,(),"[{'entity': RICS, 'label': 'SKILL'}, {'entity'..."


In [3]:
new_data = data.drop_duplicates(subset ="position").reset_index()
new_data.to_csv(PATH + '/skills_matcher/data/data_final_raw_new.csv')

In [5]:
def extract_skills_auto(s):
    words_pattern = '[a-z]+'
    return re.findall(words_pattern, s, flags=re.IGNORECASE)

In [6]:
import re

#  new with non word char inside entities
def extract_entities_2(s):
    pattern = re.compile("'entity': [a-zA-Z]+\W*([a-zA-Z]|\s)*")
    res = re.finditer(pattern, s)
    return [ mo.group().split(",")[0].strip().lstrip("'entity': ") for mo in res if mo.group() != ',']

def extract_lables(s):
    pattern = re.compile("'label':\s*'\w+'")
    res = re.finditer(pattern, s)
    return [ mo.group().partition("': '")[2].strip("'") for mo in res]

In [7]:
def get_dict(x):
    
    dict_ = {}
    
    entities = extract_entities_2(x['entities_manual_label'])
    labels = extract_lables(x['entities_manual_label'])
    for index, key in enumerate(entities):
        if key not in dict_.keys():
            dict_[key] = labels[index]

    skills_auto =extract_skills_auto(x['entities_auto_label'])
    for index, value in enumerate(skills_auto):
        if value not in dict_.keys():
            dict_[value] = 'SKILL'
    
    skills_list = []
    knowledge_list = []
    min_exp_list = []
    level_list = []
            
    for entity, label in dict_.items():
        if label == 'SKILL':
            skills_list.append(entity)
        if label == 'KNOWLEDGE':
            knowledge_list.append(entity)
        if label == 'MIN_EXP':
             min_exp_list.append(entity)
        if label == 'LEVEL':
            level_list.append(entity)
    
    x["SKILL"] = skills_list
    x["KNOWLEDGE"] = knowledge_list
    x["MIN_EXP"] = min_exp_list
    x["LEVEL"] = level_list
    
    return x

In [163]:
data_cleaned = new_data.apply(get_dict, axis=1)

In [175]:
def lowercase(s):
    new_list = [skill.lower() for skill in s]
    return new_list

def remove_punktuation(s):
    new_list = [re.sub(r'[^\w\s+]', '', skill).replace('\xa0', '') for skill in s]
    return new_list

In [174]:
data_cleaned['SKILL'] = data_cleaned['SKILL'].apply(lowercase).apply(remove_punktuation)
data_cleaned['KNOWLEDGE'] = data_cleaned['KNOWLEDGE'].apply(lowercase).apply(remove_punktuation)
data_cleaned['MIN_EXP'] = data_cleaned['MIN_EXP'].apply(lowercase).apply(remove_punktuation)
data_cleaned['LEVEL'] = data_cleaned['LEVEL'].apply(lowercase).apply(remove_punktuation)

In [172]:
def print_final_table(position, data):
    
    index = data_cleaned.index[data_cleaned['position'] == position].to_list()
    
    list_skills = data_cleaned['SKILL'][index].to_list()[0]
    list_knowledge = data_cleaned['KNOWLEDGE'][index].to_list()[0]
    list_min_exp = data_cleaned['MIN_EXP'][index].to_list()[0]
    list_level = data_cleaned['LEVEL'][index].to_list()[0]
    print(list_skills)
    
    return pd.DataFrame({'skills': pd.Series(list_skills), 
                         'knowledge': pd.Series(list_knowledge),
                         'level': pd.Series(list_knowledge,
                         'min_exp': pd.Series(list_knowledge))})

In [173]:
print_final_table('Accelerator Physicist id54315', data_cleaned)

['English', 'German']


,skills,knowledge
0,English,physics
1,German,NaN


In [183]:
# sums = []
# for index, row in enumerate(data_cleaned['SKILL'] + data_cleaned['KNOWLEDGE'] + data_cleaned['MIN_EXP'] + data_cleaned['LEVEL']):
#     sums.append(len((data_cleaned['SKILL'] + data_cleaned['KNOWLEDGE'] + data_cleaned['MIN_EXP'] + data_cleaned['LEVEL'][index])))
#     
# data_cleaned['number_entities'] = sums